# EVALUACIÓN MODULO 2 SPRINT 2
### Paloma Mesón

OBJETIVO: Identificar todas las universidades ubicadas en tres países específicos: Estados Unidos, Canadá y Argentina.

In [1]:
# importamos las librerias
import pandas as pd
import sys

sys.path.append("../")
from src import back_universities as bu

### EXTRACCIÓN Y LIMPIEZA DE DATOS

1. Crearemos nuestro dataframe con los datos extraidos de la API  "Universities Hipolabs".

In [2]:
# creamos una lista con los paises de los que queremos extraer los datos de las universidades
country_list = ['United States', 'Canada', 'Argentina']

In [3]:
# llamamos a la clase de extracción y creamos el dataframe
ext = bu.Extraction(country_list)
df_uni = ext.create_dataframe()

In [4]:
# Comprobamos que la información se ha descargado correctamente
df_uni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522 entries, 0 to 2521
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state-province  273 non-null    object
 1   domains         2522 non-null   object
 2   name            2522 non-null   object
 3   country         2522 non-null   object
 4   web_pages       2522 non-null   object
 5   alpha_two_code  2522 non-null   object
dtypes: object(6)
memory usage: 118.3+ KB


In [5]:
# procedemos a la limpieza del dataframe, para ello primero debemos llamar a la clase
clean = bu.Clean_Dataframe(df_uni) 

In [6]:
# y pasamos la funcion de limpieza definida en nuestra clase, sobreescribiendo el dataframe
df_uni = clean.clean_df()

In [7]:
# comprobamos que se ha limpiado correctamente
df_uni.sample(3)

,country,state_province,name,web_pages,alpha_two_code
847,United States,Uknown,South Texas College of Law,http://www.stcl.edu/,US
585,United States,Uknown,Marymount Manhattan College,http://www.mmm.edu/,US
264,United States,Uknown,Central Connecticut State University,http://www.ccsu.edu/,US


In [8]:
df_uni['state_province'].unique()

array(['Uknown', 'Pennsylvania', 'Nevada', 'Iowa', 'Virginia', 'Texas',
       'Colorado', 'Indicanapolis', 'California', 'South Carolina',
       'Washington', 'New York', 'North Dacota', 'Michigan', 'Ohio',
       'Florida', 'North Carolina', 'Georgia', 'Quebec', 'Ontario',
       'Nova Scotia', 'British Columbia', 'Alberta', 'Manitoba',
       'New Brunswick', 'Saskatchewan', 'Newfoundland and Labrador',
       'Prince Edward Island', 'Yukon', 'Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

In [9]:
# ahora añadiremos una columna con la latitud y la longitud para completar la información. Nuevamente, sobreescribimos el dataframe:
df_uni = clean.lat_lon('paloma')

In [10]:
# comprobamos que los datos se han actualizado correctamente
df_uni[df_uni['coordinates'] != 'Unknown'].sample(3)

,country,state_province,name,web_pages,alpha_two_code,coordinates
2346,Canada,Ontario,Ontario College of Art and Design,http://www.ocad.ca/,CA,"(50.000678, -86.000977)"
2442,Argentina,Buenos Aires,Instituto Universitario Aeronáutico,http://www.iua.edu.ar/,AR,"(-34.6075682, -58.4370894)"
2520,Argentina,Buenos Aires,Universidad Abierta Interamericana,http://www.vaneduc.edu.ar/uai/,AR,"(-34.6075682, -58.4370894)"


In [11]:
# Por último, guardamos los datos en un csv.
clean.save_df('df_uni')

--------------------------------------------------------------------------------------------

### CREACIÓN DE BASES DE DATOS E INSERCIÓN DE DATOS

In [12]:
# llamamos a la clase de creación en inserción de datos en DDBB
ddbb = bu.Create_Database() 

In [13]:
# creamos la base de datos
ddbb.create_ddbb()

Connected to MySQL
Data Base created correctly


In [14]:
# creamos las tablas utilizando las queries guardadas en nuestro documento de soporte
ddbb.insert_table(bu.query_country)
ddbb.insert_table(bu.query_universities)

In [15]:
# insertamos primero los datos de la tabla de los paises/estados debido a que no tiene Foreign Keys
list_states_final = []
# creamos una lista vacía que nos servirá para llevar el conteo de los estados incluidos en la bbdd y así no repetirlos, dado que queremos que cada uno esté asociado a un ID

for indice, fila in df_uni.iterrows():
    
    if len(list_states_final) == 0 or fila['state_province'] not in list_states_final:
        if 'U' in fila['coordinates']:
            query_uknown = f"""
                INSERT INTO countries (country_name, state_name, latitud, longitud)
                VALUES ( "{fila['country']}", "{fila['state_province']}", "0", "0");
                """
            ddbb.insert_table(query_uknown)
            list_states_final.append('Uknown')
        
        else:
            query_country = f"""
                    INSERT INTO countries (country_name, state_name, latitud, longitud)
                    VALUES ( "{fila['country']}", "{fila['state_province']}", "{float(fila['coordinates'][0])}", "{float(fila['coordinates'][1])}");
                    """
            ddbb.insert_table(query_country)
            list_states_final.append(fila['state_province'])
    else:
        pass

In [16]:
# antes de insertar los datos de la segunda tabla, necesitamos extraer los IDs asociados a cada estado para poder incluirlos como Foreign Key en la tabla con las universidades
dict_idstates = {}
# En un diccionario nueco iremos asociando estado a ID. Para iterar, utilizaremos la lista creada en el apartado anterior, con los nombres de cada Estado no repetidos
for state in list_states_final:
    id_state = ddbb.get_idstate(state)
    dict_idstates[state] = id_state

In [17]:
# una vez tenemos el diccionario, lo podemos convertir a dataframe para después unirlo con nuestro dataframe
df_idstate = pd.DataFrame.from_dict(dict_idstates, orient='index', columns=['idstate']).reset_index()

In [18]:
# actualizamos el dataframe y eliminamos las columnas que no nos interesan y limpiamos las columnas con errores
df_final = df_uni.merge(df_idstate, left_on= 'state_province', right_on= 'index', how = 'left' )
df_final.drop(columns = 'index', inplace = True)
df_final['name'] = df_final['name'].apply(clean.clean_univesity_name)

In [19]:
df_final.sample(5)

,country,state_province,name,web_pages,alpha_two_code,coordinates,idstate
2292,Canada,British Columbia,College of New Caledonia,http://www.cnc.bc.ca/,CA,"(55.001251, -125.002441)",22.0
1202,United States,Uknown,Western State Colorado University,http://www.western.edu/,US,Unknown,1.0
1398,United States,Uknown,Reedley College,http://www.reedleycollege.edu,US,Unknown,1.0
1941,United States,Uknown,Chemeketa Community College,http://www.chemeketa.edu,US,Unknown,1.0
352,United States,Uknown,DeVry Institute of Technology,http://www.devry.edu/,US,Unknown,1.0


In [20]:
# Finalmente añadimos los datos de la segunda tabla. 
# Como MySQL Workbench no nos permite subir más de 1000 datos a nuestra tabla, optamos por prioridar aquellas de las que conocemos la provincia
for _ , fila in df_final.iterrows():  
    if fila['idstate'] != 1:
        query_university = f"""
            INSERT INTO university (university_name, web_page, idstate)
            VALUES ("{fila['name']}", "{fila['web_pages']}", "{fila['idstate']}");
                """
        ddbb.insert_table(query_university)
    else:
        pass

1366 (HY000): Incorrect integer value: 'nan' for column 'idstate' at row 1
Error Code: 1366
SQLSTATE HY000
Message Incorrect integer value: 'nan' for column 'idstate' at row 1


In [21]:
# Y añadimos el resto de datos
for _, fila in df_final.iterrows():  
    if fila['idstate'] == 1:
        query_university = f"""
            INSERT INTO university (university_name, web_page, idstate)
            VALUES ("{fila['name']}", "{fila['web_pages']}", "{fila['idstate']}");
                """
        ddbb.insert_table(query_university)
    else:
        pass

-----------------------------------------------------------